In [1]:
import keras.applications.inception_v3 as googlenet
from keras.layers import Dropout, Flatten, Dense
import numpy as np

#이미지 생성용
from keras.preprocessing.image import ImageDataGenerator
from collections import Counter

# 콜백 함수
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

#시간 구하기용
from datetime import datetime

#디렉토리 생성용
import os

Using TensorFlow backend.


In [2]:
# 설정값들
IMAGE_SIZE = 128
IMAGE_DEPTH = 3
LABEL_NUM = 16

# 이미지 volume 값
input_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_DEPTH)

#GetCNN Model
model = googlenet.InceptionV3(weights=None, input_shape=input_shape, pooling='max', classes=LABEL_NUM)

# 모델 출력
model.summary()

#Compling CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 63, 63, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 63, 63, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 63, 63, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [3]:
# Val값에 따라 훈련 조기종료 설정
early_stopping = EarlyStopping(patience=2)

# Class Weights 계산
# 가장 많은 클레스의 수 / 각 클레스의 수
# 클레스 = 각 라벨에 해당하는 데이터 수
'''
counter = Counter(training_set.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
'''

#오늘 날짜_시간
NOW = datetime.now().strftime('%Y%m%d_%H_%M_%S')

#텐서보드 로그
tensorboard = TensorBoard(log_dir="./Logs/GoogLeNet/" + NOW)

#디렉토리 생성
def MakeDirectory(dir_path):
    try:
        if not(os.path.isdir(dir_path)):
            os.makedirs(os.path.join(dir_path))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

In [4]:
#훈련 6개의 데이터 셋으로 진행(Validation 데이터 셋이 다름)
DATASET_NUM = 6
EPOCH_NUM = 20
BATCH_SIZE = 32
CYCLE_NUM = 8

#훈련용 이미지 생성기
train_datagen = ImageDataGenerator(horizontal_flip = True)
validation_datagen = ImageDataGenerator(horizontal_flip = True)

# Weight, Model 저장용 폴더 생성
model_save_dir = "./Models/GoogLeNet/" + NOW
MakeDirectory(model_save_dir)

#모델 저장
model_json = model.to_json()
with open(model_save_dir + "/model_googlenet_" + NOW + ".json", "w") as json_file : 
    json_file.write(model_json)
             
#몇 사이클 돌지
for cycle in range(0, CYCLE_NUM):
    
    print("Cycle ", (cycle + 1))
    
    # 데이터셋 수 만큼 훈련
    for index in range(1, (DATASET_NUM + 1)):

        print("Training ", index)

        # 데이터 셋 생성
        dir_path = "".join(['./Dataset_Multi/Dataset', str(index)])
        training_set = train_datagen.flow_from_directory(dir_path + '/Train',
                                                    target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                    batch_size = BATCH_SIZE)

        validation_set = validation_datagen.flow_from_directory(dir_path + '/Validation',
                                                   target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                   batch_size = BATCH_SIZE)

        #배치 수 계산
        train_step_epoch = int(len(training_set.classes) / BATCH_SIZE)
        validation_step_epoch = int(len(validation_set.classes) / BATCH_SIZE)
        
        #훈련!
        model.fit_generator(training_set,
                            steps_per_epoch = train_step_epoch,
                            epochs = EPOCH_NUM,
                            validation_data = validation_set,
                            validation_steps = validation_step_epoch,
                            #class_weight=class_weights,
                            shuffle=True,
                            callbacks=[tensorboard, early_stopping])

        print("Traing ", index, " done")
        
        #오늘 날짜_시간
        current_time = datetime.now().strftime('%Y%m%d_%H_%M_%S')
        
        # Weight 저장
        model.save_weights(model_save_dir + "/weight_googlenet_" + current_time + ".h5")
        print("Saved model to disk\n")

Cycle  1
Training  1
Found 62573 images belonging to 16 classes.
Found 15875 images belonging to 16 classes.
Epoch 1/20
1955/1955 [==============================] - 876s 448ms/step - loss: 2.8715 - acc: 0.2546 - val_loss: 6.1939 - val_acc: 0.2138
Epoch 2/20
1955/1955 [==============================] - 247s 126ms/step - loss: 2.7420 - acc: 0.2578 - val_loss: 2.7002 - val_acc: 0.2533
Epoch 3/20
1955/1955 [==============================] - 246s 126ms/step - loss: 2.4793 - acc: 0.2603 - val_loss: 3.1336 - val_acc: 0.2330
Epoch 4/20
1955/1955 [==============================] - 247s 126ms/step - loss: 2.4631 - acc: 0.2619 - val_loss: 2.4569 - val_acc: 0.2536
Epoch 5/20
1955/1955 [==============================] - 246s 126ms/step - loss: 2.4734 - acc: 0.2612 - val_loss: 2.8405 - val_acc: 0.2530
Epoch 6/20
1955/1955 [==============================] - 246s 126ms/step - loss: 2.4679 - acc: 0.2617 - val_loss: 2.5715 - val_acc: 0.2517
Traing  1  done
Saved model to disk

Training  2
Found 68806 im

1955/1955 [==============================] - 247s 126ms/step - loss: 2.0968 - acc: 0.3188 - val_loss: 2.2153 - val_acc: 0.2891
Epoch 2/20
1955/1955 [==============================] - 247s 126ms/step - loss: 2.0767 - acc: 0.3234 - val_loss: 2.2046 - val_acc: 0.2958
Epoch 3/20
1955/1955 [==============================] - 247s 126ms/step - loss: 2.0688 - acc: 0.3291 - val_loss: 2.2277 - val_acc: 0.2851
Epoch 4/20
1955/1955 [==============================] - 247s 126ms/step - loss: 2.1261 - acc: 0.3154 - val_loss: 2.2101 - val_acc: 0.2850
Traing  1  done
Saved model to disk

Training  2
Found 68806 images belonging to 16 classes.
Found 9642 images belonging to 16 classes.
Epoch 1/20
2150/2150 [==============================] - 265s 123ms/step - loss: 2.0532 - acc: 0.3304 - val_loss: 2.1178 - val_acc: 0.3127
Epoch 2/20
2150/2150 [==============================] - 265s 123ms/step - loss: 2.0363 - acc: 0.3356 - val_loss: 2.1302 - val_acc: 0.3096
Epoch 3/20
2150/2150 [=========================

2150/2150 [==============================] - 266s 124ms/step - loss: 1.2733 - acc: 0.5663 - val_loss: 1.2130 - val_acc: 0.5878
Traing  2  done
Saved model to disk

Training  3
Found 62272 images belonging to 16 classes.
Found 16176 images belonging to 16 classes.
Epoch 1/20
1946/1946 [==============================] - 246s 127ms/step - loss: 1.2162 - acc: 0.5866 - val_loss: 1.1638 - val_acc: 0.6025
Epoch 2/20
1946/1946 [==============================] - 246s 127ms/step - loss: 1.1844 - acc: 0.5986 - val_loss: 1.1000 - val_acc: 0.6163
Epoch 3/20
1946/1946 [==============================] - 246s 127ms/step - loss: 1.1457 - acc: 0.6109 - val_loss: 1.3677 - val_acc: 0.5473
Epoch 4/20
1946/1946 [==============================] - 246s 127ms/step - loss: 1.0868 - acc: 0.6305 - val_loss: 1.3723 - val_acc: 0.5464
Traing  3  done
Saved model to disk

Training  4
Found 64149 images belonging to 16 classes.
Found 14299 images belonging to 16 classes.
Epoch 1/20
2004/2004 [=========================

Found 14299 images belonging to 16 classes.
Epoch 1/20
2004/2004 [==============================] - 252s 126ms/step - loss: 0.4645 - acc: 0.8421 - val_loss: 0.4034 - val_acc: 0.9474
Epoch 2/20
2004/2004 [==============================] - 251s 125ms/step - loss: 0.3994 - acc: 0.8652 - val_loss: 0.1772 - val_acc: 0.9572
Epoch 3/20
2004/2004 [==============================] - 251s 125ms/step - loss: 0.4061 - acc: 0.8622 - val_loss: 0.5126 - val_acc: 0.9292
Epoch 4/20
2004/2004 [==============================] - 253s 126ms/step - loss: 0.3750 - acc: 0.8721 - val_loss: 0.2960 - val_acc: 0.9222
Traing  4  done
Saved model to disk

Training  5
Found 65987 images belonging to 16 classes.
Found 12461 images belonging to 16 classes.
Epoch 1/20
2062/2062 [==============================] - 257s 125ms/step - loss: 0.4004 - acc: 0.8619 - val_loss: 0.5873 - val_acc: 0.9287
Epoch 2/20
2062/2062 [==============================] - 257s 125ms/step - loss: 0.3878 - acc: 0.8685 - val_loss: 0.6041 - val_acc

In [6]:
#테스트셋 불러오기
test_datagen = ImageDataGenerator(horizontal_flip = True)
test_set = validation_datagen.flow_from_directory('./Dataset/Test',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(
            test_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")
output = model.predict_generator(test_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_set.class_indices)
print(output[0])

Found 9181 images belonging to 16 classes.
-- Evaluate --
acc: 86.44%
-- Predict --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.958 0.000 0.000 0.000
 0.000 0.000 0.042 0.000]


In [ ]:
# 모델 평가하기 - 트레이닝 셋으로
print("-- Evaluate --")
scores = model.evaluate_generator(
            training_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기 - 트레이닝 셋으로
print("-- Predict --")
output = model.predict_generator(training_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(training_set.class_indices)
print(output[0])